In [5]:
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util

%matplotlib inline
import matplotlib.pyplot as plt

from coffea.nanoaod import NanoEvents

from pyinstrument import Profiler
profiler = Profiler()
profiler.start()

from coffea.nanoaod import NanoEvents
from coffea.nanoaod.methods import FatJet, LorentzVector

arraycache = {}
events = NanoEvents.from_file(
    'data/tt-semilep/nano_mc_2017_0.root',
    #'data/hww_2017mc_UL-hadd/nano_RunIISummer19UL17MiniAOD_Feb10.root',
    cache=arraycache,
    entrystart=100000,
    entrystop=500000,
    methods={
        'FatJetPFCands': LorentzVector,
    },
)
print(arraycache)

profiler.stop()

{}


In [56]:
def getParticles(events,id=25,flags=['fromHardProcess', 'isLastCopy'],status=None):
    absid = np.abs(events.GenPart.pdgId)
    if isinstance(id,int):
        idx = (absid == id) & events.GenPart.hasFlags(flags)
    else:
        idx = (absid >= id[0]) & (absid <= id[1]) & events.GenPart.hasFlags(flags)
    if status:
        idx = idx & events.GenPart.status == status
    return events.GenPart[idx],idx

def match(left, right, metric, maximum=np.inf):
    '''Matching utility                                                                                                                                                              
    For each item in ``left``, find closest item in ``right``, using function ``metric``.                                                                                            
    The function must accept two broadcast-compatible arrays and return a numeric array.                                                                                             
    If maximum is specified, mask matched elements where metric was greater than it.                                                                                                 
    '''
    lr = left.cross(right, nested=True)
    mval = metric(lr.i0, lr.i1)
    idx = mval.argmin()
    print(mval,idx)
    if maximum < np.inf:
        matched = lr.i1[idx[mval[idx] < maximum]]
        return matched.copy(content=matched.content.pad(1)).flatten(axis=1)
    else:
        return lr.i1[idx]
    
def getFlavor(childid, momid=None):
    x = ((childid == 24).any() * 6 + (childid == 13).any() * 5 + (childid == 11).any() * 4 + (childid == 5).any() * 3 + (childid == 4).any() * 2 + (childid < 4).all() * 1).pad(1, clip=True).fillna(0).flatten()
    x = x + ((momid == 5).any() * 7)
    return x
    
def matchedParticleFlavor(candidates, particles, types='child', maxdR=0.8): 
    matched = match(candidates, particles, lambda a, b: a.delta_r(b), maxdR)
    if types=='child':
        retid = abs(matched.children.pdgId)
    else:
        retid = abs(matched.pdgId)
    return retid

leadingmuon =  events.Muon[:,0:1]
goodFatJet = (
    (events.FatJet.pt > 300.)
    & (np.abs(events.FatJet.eta) < 2.4)
    & (events.FatJet.msoftdrop > 10.)
    & (events.FatJet.isTight)
            )
fatjets = events.FatJet[goodFatJet]
ak8_muon_pair = leadingmuon.cross(fatjets)
ak8_muon_dR = ak8_muon_pair.i0.delta_r(ak8_muon_pair.i1)
candidatejet = fatjets[ak8_muon_dR.argmin()]

genT,genT_idx = getParticles(events,6,['fromHardProcess', 'isLastCopy'])
genW,genW_idx = getParticles(events,24,['fromHardProcess', 'isLastCopy'])
genb,genb_idx = getParticles(events,5,['fromHardProcess', 'isLastCopy'])

genflavorW = matchedParticleFlavor(leadingmuon, genW,'child', 0.2)
genflavorb = matchedParticleFlavor(leadingmuon, genb,'mom', 0.2)
genflav = getFlavor(genflavorW,genflavorb)
print('r',genflav)


[[] [] [] ... [[2.927546 1.5027124]] [[1.905976 1.394493]] []] [[] [] [] ... [[1]] [[1]] []]
[[] [] [] ... [[0.96428144 2.1354272]] [[3.2282088 0.025006808]] []] [[] [] [] ... [[0]] [[1]] []]
r [0 0 0 ... 0 7 0]


In [ ]:
def getParticles(events,id=25,flags=['fromHardProcess', 'isLastCopy'],status=None):
    absid = np.abs(events.GenPart.pdgId)
    if isinstance(id,int):
        idx = (absid == id) & events.GenPart.hasFlags(flags)
    else:
        idx = (absid >= id[0]) & (absid <= id[1]) & events.GenPart.hasFlags(flags)
    if status:
        idx = idx & events.GenPart.status == status
    return events.GenPart[idx],idx

genH,genH_idx = getParticles(events,25,['fromHardProcess', 'isLastCopy'])
genW,genW_idx = getParticles(events,24,['fromHardProcess', 'isLastCopy'])
genE,genE_idx = getParticles(events,11,['fromHardProcess','isFirstCopy'],1)
genM,genM_idx = getParticles(events,13,['fromHardProcess','isFirstCopy'],1)
genT,genT_idx = getParticles(events,15,['fromHardProcess','isFirstCopy'],1)
genQ,genQ_idx = getParticles(events,[0,5],['fromHardProcess','isFirstCopy'])

ishWW_qqelev = (genH.counts==1) & (genW.counts==2) & (genE.counts==1) & (genM.counts==0) & (genT.counts==0)
ishWW_qqmuv = (genH.counts==1) & (genW.counts==2) & (genM.counts==1) & (genE.counts==0) & (genT.counts==0)
ishWW_qqtauv = (genH.counts==1) & (genW.counts==2) & (genT.counts==1) & (genM.counts==0) & (genE.counts==0)
ishWW_qqqq = (genH.counts==1) & (genW.counts==2) & (genQ.counts==4)  & (genM.counts==0) & (genE.counts==0)
ishWW_muvelev = (genH.counts==1) & (genW.counts==2) & (genE.counts==1) & (genM.counts==1)
ishWW_elevelev = (genH.counts==1) & (genW.counts==2) & (genE.counts==2) & (genM.counts==0)
ishWW_tauvtauv = (genH.counts==1) & (genW.counts==2) & (genT.counts==2) & (genM.counts==0) & (genE.counts==0)
ishWW_muvmuv = (genH.counts==1) & (genW.counts==2) & (genE.counts==0) & (genM.counts==2)


In [ ]:
absid = np.abs(events.GenPart.pdgId)[1]
idx = (absid >= 0) & (absid <= 5) 
flags = events.GenPart.hasFlags(['fromHardProcess','isFirstCopy'])[1][idx]
print(absid)
print(idx)
print(events.GenPart.hasFlags(['fromHardProcess','isFirstCopy'])[1][idx])
print(events.GenPart.status[1][idx][flags])
print(genQ.counts[1])
print(genM.counts[1])
print(genE.counts[1])
print(genW.counts[1])
print(~ishWW_qqmuv[1])
print(~ishWW_qqqq[1])
print(((~ishWW_qqqq) & (~ishWW_qqmuv) & (~ishWW_qqelev) & (~ishWW_muvelev) & (~ishWW_elevelev) & (~ishWW_muvmuv) & (~ishWW_tauvtauv) & (~ishWW_qqtauv)).any())
#print(len(events))
#124226+ 47877 +48135 +3240 + 6608 + 3344

goodFatJet = (
    (events.FatJet.pt > 300.)
    & (np.abs(events.FatJet.eta) < 2.4)
    & (events.FatJet.msoftdrop > 10.)
    #& (events.FatJet.jetId & 2)
    & (events.FatJet.isTight)
            )
fatjets = events.FatJet[goodFatJet]
ak8_muon_pair = leadingmuon.cross(fatjets)
ak8_muon_dR = ak8_muon_pair.i0.delta_r(ak8_muon_pair.i1)
closestjet = fatjets[ak8_muon_dR.argmin()]
leadingjet = fatjets[:,0:1]

closestjet_muon_pair = leadingmuon.cross(closestjet)
closestjet_muon_dR = closestjet_muon_pair.i0.delta_r(closestjet_muon_pair.i1)

leadingjet_muon_pair = leadingmuon.cross(leadingjet)
leadingjet_muon_dR = leadingjet_muon_pair.i0.delta_r(leadingjet_muon_pair.i1)


print(len(ak8_muon_dR.min() < 0.8))
with np.printoptions(threshold=np.inf):
    print(ak8_muon_dR.tolist()[-6])
    print(ak8_muon_dR.min().tolist()[-6])
    print(ak8_muon_dR.argmin().tolist()[-6])
    print(closestjet_muon_dR.tolist()[-6])
    print(leadingjet_muon_dR.tolist()[-6])
plt.clf();
#plt.hist(ak8_muon_dR.flatten(), label='dR all jets');
plt.hist(closestjet_muon_dR.flatten(), label='dR closestjet');
#plt.hist(leadingjet_muon_dR.flatten(), label='dR leadingjet');
plt.show()
plt.clf();
plt.hist(np.nan_to_num(ak8_muon_dR.min().flatten()), label='dR min all jets');
plt.show()
plt.clf();

In [ ]:
print(closestjet[-2])
print(leadingjet[-2])
print(ak8_muon_dR.argmin()[-2])
print(goodFatJet)
print(events.FatJet[goodFatJet].eta)
print(np.abs(events.FatJet.eta))
print(np.abs(events.FatJet.eta) < 2.4)
print(events.FatJet.msoftdrop > 10)
print((np.abs(events.FatJet.eta) < 2.4) & (events.FatJet.msoftdrop > 10))
print(events.FatJet.pt > 300.)
print((np.abs(events.FatJet.eta) < 2.4) & (events.FatJet.msoftdrop > 10) & (events.FatJet.pt > 300.))
print((events.FatJet.jetId & 2))
print(events.FatJet[(np.abs(events.FatJet.eta) < 2.4) & (events.FatJet.msoftdrop > 10) & (events.FatJet.pt > 300.)].eta)
print(events.FatJet[(np.abs(events.FatJet.eta) < 2.4) & (events.FatJet.msoftdrop > 10)].eta)
print(events.FatJet[events.FatJet.msoftdrop > 10].eta)

In [ ]:
print(ak8_muon_dR,ak8_muon_dR.min(),ak8_muon_dR.argmin())
print(ak8_muon_pair)
print(closestjet_muon_pair)
print(closestjet_muon_dR)
print(ak8_muon_dR)
plt.clf();
#plt.hist(ak8_muon_dR.flatten(), label='dR all jets');
#plt.hist(closestjet_muon_dR.flatten(), label='dR closestjet');
plt.hist(np.nan_to_num(ak8_muon_dR.min().flatten()), label='dR min all jets');

plt.show()
plt.clf();


In [ ]:
plt.hist(leadingjet.muonIdx3SJ.flatten());
plt.show()
plt.clf();
plt.hist(leadingjet.LSmsoftdrop.flatten(), bins=np.linspace(0, 200, 50), alpha=0.5, label='LS msoftdrop');
plt.hist(leadingjet.msoftdrop.flatten(), bins=np.linspace(0, 200, 50), alpha=0.5, label='Jet msoftdrop');
plt.hist((leadingjet - leadingmuon).mass.flatten(), bins=np.linspace(0, 200, 50), alpha=0.5, label='Jet - muon mass');
plt.gca().legend()

In [ ]:
lscut = (events.CustomAK8Puppi.muonIdx3SJ >= 0)
events = events[lscut.any()]
lscut = lscut[lscut.any()]
ljet = events.CustomAK8Puppi[:,0:1]
print(len(ljet.muonIdx3SJ.flatten()))
mu = events.Muon[ljet.muonIdx3SJ.flatten()]
print(len(mu),len(ljet))
print(mu-ljet)
from uproot_methods import TLorentzVectorArray

leadingjet_p4 = TLorentzVectorArray.from_ptetaphim(ljet.pt, ljet.eta, ljet.phi, ljet.msoftdrop)
leadingmuon_p4 = TLorentzVectorArray.from_ptetaphim(mu.pt,mu.eta,mu.phi, mu.mass)
print(leadingjet_p4.pad(1, clip=False).flatten())
print(leadingmuon_p4.pad(1, clip=False).flatten())
mm = (leadingjet_p4 - leadingmuon_p4).mass2

#mm = (ljet - mu).mass2
print(mm)
jmass = (mm>0)*np.sqrt(np.maximum(0, mm)) + (mm<0)*leadingjet.mass
print(len(jmass.flatten()))
print(len(mu.flatten()))
plt.hist2d(jmass.flatten(), mu.pt.flatten(), bins=50);

In [ ]:
(met_p4 + leadingjet).mass

In [ ]:
print(len(events.FatJet.muonIdx3SJ.flatten()))
print((events.FatJet.muonIdx3SJ.flatten()>=0).any())
lscut = (events.FatJet.muonIdx3SJ >= 0).any()
print(events.FatJet[(events.FatJet.muonIdx3SJ >= 0).any()].flatten())
print(events.FatJet.muonIdx3SJ.flatten())
newevents = events[lscut.any()]
lscut = lscut[lscut.any()]
jet = newevents.FatJet[lscut]
print((events.FatJet.muonIdx3SJ >= 0).any())
print(jet)

print(events.FatJet[:,0:1])
mu = events.Muon[events.FatJet[:,0:1].array.muonIdx3SJ]
mm = (leadingjet - mu).mass2

In [ ]:
print(events.run)